In [1]:
%load_ext autotime
import sys
sys.path.append('/Users/pradap/Documents/Research/Python-Package/anhaid/py_entitymatching/')

In [2]:
import py_entitymatching as em

time: 1.75 s


In [3]:
A = em.load_dataset('person_table_A', key='ID')
B = em.load_dataset('person_table_B', key='ID')

time: 6.29 ms


In [4]:
A

,ID,name,birth_year,hourly_wage,address,zipcode
0,a1,Kevin Smith,1989,30.0,"607 From St, San Francisco",94107
1,a2,Michael Franklin,1988,27.5,"1652 Stockton St, San Francisco",94122
2,a3,William Bridge,1986,32.0,"3131 Webster St, San Francisco",94107
3,a4,Binto George,1987,32.5,"423 Powell St, San Francisco",94122
4,a5,Alphonse Kemper,1984,35.0,"1702 Post Street, San Francisco",94122


time: 31.4 ms


In [5]:
B

,ID,name,birth_year,hourly_wage,address,zipcode
0,b1,Mark Levene,1987,29.5,"108 Clement St, San Francisco",94107
1,b2,Bill Bridge,1986,32.0,"3131 Webster St, San Francisco",94107
2,b3,Mike Franklin,1988,27.5,"1652 Stockton St, San Francisco",94122
3,b4,Joseph Kuan,1982,26.0,"108 South Park, San Francisco",94122
4,b5,Alfons Kemper,1984,35.0,"170 Post St, Apt 4, San Francisco",94122
5,b6,Michael Brodie,1987,32.5,"133 Clement Street, San Francisco",94107


time: 14.3 ms


In [6]:
L = em.read_csv_metadata('person_labeled_data.csv', key='_id')

time: 8.55 ms


In [7]:
help(em.extract_feature_vecs)

Help on function extract_feature_vecs in module py_entitymatching.feature.extractfeatures:

extract_feature_vecs(candset, attrs_before=None, feature_table=None, attrs_after=None, verbose=False, show_progress=True)
    This function extracts feature vectors from a DataFrame (typically a
    labeled candidate set).
    
    Specifically, this function uses feature
    table, ltable and rtable (that is present in the `candset`'s
    metadata) to extract feature vectors.
    
    Args:
        candset (DataFrame): The input candidate set for which the features
            vectors should be extracted.
        attrs_before (list): The list of attributes from the input candset,
            that should be added before the feature vectors (defaults to None).
        feature_table (DataFrame): A DataFrame containing a list of
            features that should be used to compute the feature vectors (
            defaults to None).
        attrs_after (list): The list of attributes from the input c

In [8]:
em.set_fk_ltable(L, 'ltable_ID')
em.set_fk_rtable(L, 'rtable_ID')
em.set_ltable(L, A)
em.set_rtable(L, B)

F = em.get_features_for_matching(A, B)
E = em.extract_feature_vecs(L, feature_table=F)

0%      100%
[##########] | ETA: 00:00:00

time: 135 ms



Total time elapsed: 00:00:00


In [ ]:
E